# NLP Code Along

For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [4]:
data = spark.read.csv("/FileStore/tables/SMSSpamCollection",inferSchema=True,sep='\t')

In [5]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [6]:
data.show()

+-----+--------------------+
class| text|
+-----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
 spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
 spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
 spam|WINNER!! As a val...|
 spam|Had your mobile 1...|
 ham|I'm gonna be home...|
 spam|SIX chances to wi...|
 spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
 spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
 spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows

## Clean and Prepare the Data

** Create a new length feature: **

In [9]:
from pyspark.sql.functions import length

In [10]:
data = data.withColumn('length',length(data['text']))

In [11]:
data.show()

+-----+--------------------+------+
class| text|length|
+-----+--------------------+------+
 ham|Go until jurong p...| 111|
 ham|Ok lar... Joking ...| 29|
 spam|Free entry in 2 a...| 155|
 ham|U dun say so earl...| 49|
 ham|Nah I don't think...| 61|
 spam|FreeMsg Hey there...| 147|
 ham|Even my brother i...| 77|
 ham|As per your reque...| 160|
 spam|WINNER!! As a val...| 157|
 spam|Had your mobile 1...| 154|
 ham|I'm gonna be home...| 109|
 spam|SIX chances to wi...| 136|
 spam|URGENT! You have ...| 155|
 ham|I've been searchi...| 196|
 ham|I HAVE A DATE ON ...| 35|
 spam|XXXMobileMovieClu...| 149|
 ham|Oh k...i'm watchi...| 26|
 ham|Eh u remember how...| 81|
 ham|Fine if thats th...| 56|
 spam|England v Macedon...| 155|
+-----+--------------------+------+
only showing top 20 rows

In [12]:
# Pretty Clear Difference
data.groupby('class').mean().show()

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham| 71.4545266210897|
 spam|138.6706827309237|
+-----+-----------------+

## Feature Transformations

In [14]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [15]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [16]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model

We'll use Naive Bayes, but feel free to play around with this choice!

In [18]:
from pyspark.ml.classification import NaiveBayes

In [19]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [21]:
from pyspark.ml import Pipeline

In [22]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [23]:
cleaner = data_prep_pipe.fit(data)

In [24]:
clean_data = cleaner.transform(data)

### Training and Evaluation!

In [26]:
clean_data = clean_data.select(['label','features'])

In [27]:
clean_data.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13424,[7,11,31,6...|
 0.0|(13424,[0,24,297,...|
 1.0|(13424,[2,13,19,3...|
 0.0|(13424,[0,70,80,1...|
 0.0|(13424,[36,134,31...|
 1.0|(13424,[10,60,139...|
 0.0|(13424,[10,53,103...|
 0.0|(13424,[125,184,4...|
 1.0|(13424,[1,47,118,...|
 1.0|(13424,[0,1,13,27...|
 0.0|(13424,[18,43,120...|
 1.0|(13424,[8,17,37,8...|
 1.0|(13424,[13,30,47,...|
 0.0|(13424,[39,96,217...|
 0.0|(13424,[552,1697,...|
 1.0|(13424,[30,109,11...|
 0.0|(13424,[82,214,47...|
 0.0|(13424,[0,2,49,13...|
 0.0|(13424,[0,74,105,...|
 1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows

In [28]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [29]:
spam_predictor = nb.fit(training)

In [30]:
data.printSchema()

root
-- class: string (nullable = true)
-- text: string (nullable = true)
-- length: integer (nullable = true)

In [31]:
test_results = spam_predictor.transform(testing)

In [32]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,2,13,...|[-621.58788098456...|[0.99999999999224...| 0.0|
 0.0|(13424,[0,1,5,15,...|[-999.96942708762...|[1.0,1.0537921777...| 0.0|
 0.0|(13424,[0,1,7,8,1...|[-1154.8758524239...|[1.0,1.9325043972...| 0.0|
 0.0|(13424,[0,1,7,15,...|[-661.26632905300...|[1.0,1.5670546264...| 0.0|
 0.0|(13424,[0,1,9,14,...|[-539.85073461691...|[1.0,3.1193568823...| 0.0|
 0.0|(13424,[0,1,14,18...|[-1373.8494984186...|[1.0,6.5648344746...| 0.0|
 0.0|(13424,[0,1,14,31...|[-215.65995389761...|[1.0,2.0054433702...| 0.0|
 0.0|(13424,[0,1,15,20...|[-687.14376672955...|[1.0,2.5607725578...| 0.0|
 0.0|(13424,[0,1,18,20...|[-872.59901895844...|[1.0,1.2946588930...| 0.0|
 0.0|(13424,[0,1,23,63...|[-1315.5298109147...|[1.0,4.6424877609...| 0.0|
 0.0|(13424,[0,1,46,17...|[-1136.3200923191...|[6.08847977107979...| 1.0|
 0.0|(13424,[0,2,3,4,6...|[-1273.7479204993...|[1.0,5.0144907753...| 0.0|
 0.0|(13424,[0,2,3,5,6...|[-2553.3908140671...|[1.0,6.2771516253...| 0.0|
 0.0|(13424,[0,2,3,5,3...|[-504.50532643271...|[1.0,1.5847330230...| 0.0|
 0.0|(13424,[0,2,3,8,2...|[-1626.4589318607...|[1.0,2.6135455784...| 0.0|
 0.0|(13424,[0,2,4,5,7...|[-988.12921186741...|[1.0,2.1899914262...| 0.0|
 0.0|(13424,[0,2,4,5,1...|[-1631.3275776624...|[1.0,2.5326870929...| 0.0|
 0.0|(13424,[0,2,4,11,...|[-1128.1095008053...|[1.0,5.0164683269...| 0.0|
 0.0|(13424,[0,2,5,8,4...|[-831.05576053348...|[1.0,1.5033289080...| 0.0|
 0.0|(13424,[0,2,7,11,...|[-1330.2526496148...|[1.0,1.0568189461...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [34]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9213461716121458

Not bad considering we're using straight math on text data! Try switching out the classification models! Or even try to come up with other engineered features!

## Great Job!